In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/RUSTEYE/dataset_cnn_v2.zip

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 80

In [3]:
train_dir = '/content/dataset_cnn_v2/train'
val_dir   = '/content/dataset_cnn_v2/valid'
test_dir = '/content/dataset_cnn_v2/test'

# Data augmentation cho train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 945 images belonging to 4 classes.
Found 202 images belonging to 4 classes.
Found 205 images belonging to 4 classes.


In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

# Thêm các layer phía trên
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# Compile
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

In [7]:
# Train
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.3331 - loss: 1.8100 - val_accuracy: 0.4059 - val_loss: 1.3702
Epoch 2/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 706ms/step - accuracy: 0.4131 - loss: 1.2402 - val_accuracy: 0.5347 - val_loss: 1.0801
Epoch 3/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 42s 725ms/step - accuracy: 0.4884 - loss: 1.1028 - val_accuracy: 0.5347 - val_loss: 0.9752
Epoch 4/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 20s 656ms/step - accuracy: 0.5425 - loss: 1.0296 - val_accuracy: 0.5050 - val_loss: 1.0245
Epoch 5/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 23s 730ms/step - accuracy: 0.5000 - loss: 1.0546 - val_accuracy: 0.5941 - val_loss: 0.8942
Epoch 6/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 694ms/step - accuracy: 0.5496 - loss: 0.9496 - val_accuracy: 0.5644 - val_loss: 0.8658
Epoch 7/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 41s 709ms/step - accuracy: 0.5559 - loss: 0.9868 - val_accuracy: 0.5248 - val_loss: 0.9404
Epoch 8/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 20s 661ms/step - accuracy: 0.5676 - loss: 0.9233 - val_accurac

In [8]:
# Load best model
from tensorflow.keras.models import load_model
best_model = load_model('best_model.keras')

In [9]:
# Đánh giá trên test set
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Lấy nhãn class
class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Accuracy theo từng class
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 718ms/step - accuracy: 0.5925 - loss: 0.8352

Test Loss: 0.6879
Test Accuracy: 70.73%
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 775ms/step
Accuracy: 70.73170731707317
Precision: 70.51864500566309
Recall: 70.73170731707317
F1 Score: 69.23904566532259

Classification Report:
                precision    recall  f1-score   support

 bong-troc-son     0.6786    0.3725    0.4810        51
day-cap-ri-set     0.6615    0.7544    0.7049        57
        ri-set     0.6452    0.7692    0.7018        52
       vet-nut     0.8600    0.9556    0.9053        45

      accuracy                         0.7073       205
     macro avg     0.7113    0.7129    0.6982       205
  weighted avg     0.7052    0.7073    0.6924       205


Accuracy for each class:
bong-troc-son: 0.37
day-cap-ri-set: 0.75
ri-set: 0.77
vet-nut: 0.96
